# FindMyWhale Localization Demo Notebook

## Overview

This notebook accesses data via 


## Table of Contents





In [1]:
# ------------------------------------------------- #
#                      MODULES                      #

# General Modules
import os
import sys
import pandas as pd
import numpy as np

sys.path.append(os.path.abspath(".."))

import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

#
from findmywhale import data_collect

#                                                   #
# ------------------------------------------------- #

In [ ]:
'/Users/tylerstevenson/Documents/FindMyWhale/data/

'/Users/tylerstevenson/Documents/FindMyWhale/examples'

In [ ]:
required_columns = {
    "type",
    "created",
    "trusted",
    "latitude",
    "longitude",
    "no_sighted",
}

In [45]:
list(required_columns)

['trusted', 'longitude', 'latitude', 'type', 'created', 'no_sighted']

In [ ]:
# Build HLS Metadata Lookup
hls_lookup = data_collect.get_hls_folders(bucket_name="audio-orcasound-net")

In [ ]:
hls_lookup["DATE"] = hls_lookup["DATETIME"].dt.date.astype(str)
hls_lookup["VALUE"] = 1

hls_lookup_pvt = (
    pd.pivot_table(hls_lookup, index="LOCATION", columns="DATE", values="VALUE")
    .fillna(0)
    .astype(int)
)

In [ ]:
hls_lookup_pvt = (
    hls_lookup_pvt.unstack().reset_index().rename(columns={0: "AVAILABILITY"})
)

In [ ]:
hls_lookup_pvt

In [ ]:
df_year = hls_lookup_pvt[hls_lookup_pvt["YEAR"] == year].reset_index(drop=True)

In [ ]:
tmp = pd.pivot_table(df_year, index="LOCATION", columns="DATE", values="AVAILABILITY")

In [ ]:
# # Extract year for grouping
# hls_lookup_pvt["YEAR"] = pd.to_datetime(hls_lookup_pvt["DATE"]).dt.year

# locations = sorted(hls_lookup_pvt["LOCATION"].unique())
# n_locs = len(locations)

# # Set up matplotlib subplots: one row per year, 1 column
# fig, axes = plt.subplots(n_locs, 1, figsize=(15, 1 * n_locs), sharex=True)

# if n_locs == 1:
#     axes = [axes]  # Make iterable if only one subplot

# for ax, location in zip(axes, locations):
#     df_year = hls_lookup_pvt[hls_lookup_pvt["LOCATION"] == location]
#     df_year["DATE"] = df_year["DATE"].str[5:].str.replace("-", "/")

#     # Pivot to matrix: rows=LOCATION, columns=DATE, values=AVAILABILITY
#     heatmap_data = df_year.pivot(index="YEAR", columns="DATE", values="AVAILABILITY")

#     # Sort columns so dates go left to right
#     heatmap_data = heatmap_data.sort_index(axis=1)

#     # Plot seaborn heatmap on the axis
#     sns.heatmap(
#         heatmap_data,
#         cmap=["lightgray", "blue"],  # Binary colors, lightgray=0, green=1
#         cbar=False,  # ax == axes[-1],          # Show colorbar only on last plot
#         ax=ax,
#         linewidths=0.5,
#         linecolor="white",
#         square=True,
#         vmin=0,
#         vmax=1,
#         # cbar_kws={'label': 'Availability'}
#     )

#     ax.set_title(f"Availability in {location}")
#     ax.set_xlabel("")
#     ax.set_ylabel("Year")
#     ax.tick_params(axis="x", labelsize=6)
#     ax.tick_params(axis="y", labelsize=6)

# # Rotate date labels on the bottom plot only
# plt.setp(axes[-1].get_xticklabels(), rotation=45, ha="right")

# plt.tight_layout()
# plt.show()

In [ ]:
# Open BC Data
import geopandas as gpd

In [ ]:
bc_gdf = gpd.read_file(
    "/Users/tylerstevenson/Documents/FindMyWhale/data/bcmca_eco_physical_ecosections_data/BCMCA_ECO_Physical_Ecosections_Data.shp"
)
# bc_gdf[bc_gdf.MARSECTION == 'Strait of Georgia'].explore()

In [ ]:
bc_gdf.explore()

In [ ]:
import h3

from shapely.geometry import shape
from shapely.geometry import Polygon

In [ ]:
# Source: https://soggy2.zoology.ubc.ca/geonetwork/srv/eng/catalog.search#/metadata/e1da07ca-353d-4b8a-a08b-643885e89e3b
bc_gdf = gpd.read_file(
    "/Users/tylerstevenson/Documents/FindMyWhale/data/ssea_regions/ssea_regions.shp"
)
bc_gdf = bc_gdf.dissolve()

bc_gdf_sampled_points = bc_gdf.sample_points(size=500, method="uniform")
bc_gdf_sampled_points = bc_gdf_sampled_points.reset_index().explode()
bc_gdf_sampled_points["latitude"], bc_gdf_sampled_points["longitude"] = zip(
    *bc_gdf_sampled_points["sampled_points"].apply(lambda x: (x.y, x.x))
)
bc_gdf_sampled_points["H3_GRID_RES3"] = bc_gdf_sampled_points.apply(
    lambda x: h3.latlng_to_cell(x.latitude, x.longitude, 3), axis=1
)

bc_gdf_sampled_points = bc_gdf_sampled_points.drop_duplicates("H3_GRID_RES3")
bc_gdf_sampled_points = bc_gdf_sampled_points[["H3_GRID_RES3"]]
bc_gdf_sampled_points["H3_CHILDREN"] = bc_gdf_sampled_points["H3_GRID_RES3"].apply(
    lambda x: h3.cell_to_children(x, 6)
)
bc_gdf_sampled_points = bc_gdf_sampled_points.explode("H3_CHILDREN")
bc_gdf_sampled_points["geometry"] = bc_gdf_sampled_points["H3_CHILDREN"].apply(
    lambda x: Polygon(shape(h3.cells_to_geo([x])))
)

In [ ]:
# bc_gdf_sampled_points

In [ ]:
bc_gdf_sampled_points = gpd.GeoDataFrame(
    bc_gdf_sampled_points[["H3_CHILDREN", "geometry"]]
)
bc_gdf_sampled_points = bc_gdf_sampled_points.set_crs("EPSG:4326")

In [ ]:
bc_gdf_sampled_points["intersects"] = bc_gdf_sampled_points["geometry"].intersects(
    bc_gdf.geometry
)

In [ ]:
bc_gdf_sampled_points[bc_gdf_sampled_points["intersects"] == True].explore()

In [ ]:
# Acartia Data
acartia_export = pd.read_csv(
    "/Users/tylerstevenson/Documents/FindMyWhale/data/ACARTIA/acartia-export.csv"
)

# Okay for now, where are things spotted?
acartia_export["created"] = acartia_export["created"].str[0:20]
acartia_export["created"] = acartia_export["created"].str.replace("T", " ")
acartia_export["DATETIME"] = pd.to_datetime(acartia_export["created"], format="mixed")
acartia_export["DATE"] = acartia_export["DATETIME"].dt.date
acartia_export = acartia_export[acartia_export.DATETIME > "2022-01-01"]
acartia_export["MONTH-YEAR"] = acartia_export["DATETIME"].apply(
    lambda x: f"{x.year}-{x.month}-01"
)

In [ ]:
acartia_export["H3_CHILDREN"] = acartia_export.apply(
    lambda x: h3.latlng_to_cell(x.latitude, x.longitude, 6), axis=1
)

In [ ]:
accart = acartia_export.groupby(["H3_CHILDREN"], as_index=False)["no_sighted"].sum()
accart = accart[accart.no_sighted < accart.no_sighted.max()]

In [ ]:
accart = pd.merge(bc_gdf_sampled_points, accart, how="left").fillna(0)
accart["no_sighted_std"] = (
    accart["no_sighted"] - accart[accart.no_sighted != 0]["no_sighted"].mean()
) / accart[accart.no_sighted != 0]["no_sighted"].std()

In [ ]:
accart[accart["intersects"] == True].explore("no_sighted_std")

In [ ]:
tmp_date = acartia_export.groupby("DATE")["no_sighted"].count().reset_index()

tmp_month_date = (
    acartia_export.groupby("MONTH-YEAR")["no_sighted"].count().reset_index()
)
tmp_month_date["DATE"] = pd.to_datetime(tmp_month_date["MONTH-YEAR"])
tmp_month_date = tmp_month_date.sort_values("DATE").reset_index(drop=True)

In [ ]:
fig = px.line(tmp_date, x="DATE", y="no_sighted")
fig.show()

In [ ]:
fig = px.line(tmp_month_date, x="DATE", y="no_sighted")
fig.show()

In [ ]:
# tmp[tmp.OBJECTID == 1].explore()

In [ ]:
# tmp.explore()

In [ ]:
# tmp[tmp.Shape_Area == tmp.Shape_Area.max()].explore()